In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
import pandas as pd
import itertools as itt
hv.notebook_extension('bokeh', width=100)
hv.archive.auto()

In [ ]:
mappings_meta_fill = pd.read_pickle("./data/mappings_meta_fill.pkl")
group_dict = dict(MS101='negative', MS104='negative', NS20='negative', NS22='negative',
                  MS102='neutral', MS103='neutral', NS24='neutral')

In [ ]:
def cell_count(df):
    rlist = []
    for ss in df['session']:
        ncell = df['session', ss].count()
        row = pd.Series(
            [ss, ncell], index=['session', 'cell_count'])
        rlist.append(row)
    return pd.concat(rlist, axis='columns', ignore_index=True).T
counts = (mappings_meta_fill.groupby(('meta', 'animal')).apply(cell_count)
          .reset_index().drop('level_1', axis='columns')
          .rename(columns={('meta', 'animal'): 'animal'})
          .infer_objects())
counts['group'] = counts['animal'].apply(lambda anm: group_dict[anm])

# diagram

![wired_valence_diagram.svg](./wired_valence_diagram.svg)

# cell count - grouped

In [ ]:
%%opts BoxWhisker [height=500, width=800]
hv_counts = hv.Dataset(counts, kdims=['session', 'group'], vdims=['cell_count'])
hv_counts.to(hv.BoxWhisker)

# cell count - individual animal with hover information

In [ ]:
%%opts Curve [height=400, width=600, tools=['hover']]
hv_counts = hv.Dataset(counts, kdims=['session', 'group', 'animal'], vdims=['cell_count'])
hv_counts.to(hv.Curve).overlay('animal').layout('group')

# overlaps - layed out as multi-level x-axis
Four types of overlaps are available for toggling:
1. overlap -- number of overlapping neurons divided by the number of union of active neurons in both sessions.
2. overlap-onA -- number of overlapping neurons divided by number of active neurons in `sessionA`.
3. overlap-onB -- number of overlapping neurons divided by number of active neurons in `sessionB`.
4. overlap-prod -- number of overlapping neurons divided by the product of active neurons in both sessions, and scaled up by the number of total active neurons for each animal.

In [ ]:
overlap_list = []
map_dict = {'1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6}
for cur_anm, cur_grp in mappings_meta_fill.groupby(mappings_meta_fill['meta', 'animal']):
    cur_ss = cur_grp['session'].dropna(axis='columns', how='all').columns
    T = cur_grp['session'].dropna(axis='rows', how='all').shape[0]
    for cur_map in itt.combinations(cur_ss, 2):
        nint = cur_grp['session'][list(cur_map)].dropna(axis='rows', how='any').shape[0]
        nuni = cur_grp['session'][list(cur_map)].dropna(axis='rows', how='all').shape[0]
        nA = cur_grp['session'][cur_map[0]].dropna().size
        nB = cur_grp['session'][cur_map[1]].dropna().size
        cur_map = tuple([map_dict[m] for m in cur_map])
        cur_ovlp = pd.Series(
            [cur_anm, cur_map[0], cur_map[1], nint/nuni, nint/nA, nint/nB, (nint*T)/(nA*nB)],
            index=['animal', 'sessionA', 'sessionB', 'overlap', 'overlap-onA', 'overlap-onB', 'overlap-prod'])
        overlap_list.append(cur_ovlp)
overlaps = pd.concat(overlap_list, axis=1, ignore_index=True).T
overlaps['group'] = overlaps['animal'].apply(lambda anm: group_dict[anm])

In [ ]:
overlaps = overlaps.melt(id_vars=['animal', 'sessionA', 'sessionB', 'group'], var_name='overlap-type', value_name='overlap-value')
overlaps['overlap-value'] = overlaps['overlap-value'].astype(float)

In [ ]:
%%opts BoxWhisker [width=1000, height=500, xrotation=90] {+framewise}
overlap_hv = hv.Dataset(overlaps, kdims=['sessionA', 'sessionB', 'group', 'overlap-type'], vdims=['overlap-value'])
overlap_hv.to(hv.BoxWhisker, kdims=['sessionA', 'sessionB', 'group'])

# overlaps - layed out as pairwise matrix of `sessionA` and `sessionB`, with individual data points and hover information
**left/blue** is **negative** group; **right/orange** is **neutral** group.

In [ ]:
%%opts BoxWhisker [width=150, height=150] {+framewise}
%%opts Scatter [tools=['hover']] (color='group', size=6, cmap='Category10') {+framewise}
overlap_hv = hv.Dataset(overlaps, kdims=['sessionA', 'sessionB', 'group', 'overlap-type'], vdims=['overlap-value', 'animal'])
box = overlap_hv.to(hv.BoxWhisker, kdims=['group'], vdims=['overlap-value'])
pts = overlap_hv.to(hv.Scatter, kdims=['group'], vdims=['overlap-value', 'animal'])
((box * pts).collate().grid(['sessionA', 'sessionB']).opts(plot=dict(shared_yaxis=True)))

In [ ]:
%%opts Curve [width=1000, height=400, xrotation=90, tools=['hover']]
overlap_hv_anm = hv.Dataset(overlaps, kdims=['animal', 'session', 'group', 'overlap-type'], vdims=['overlap-value'])
overlap_hv_anm.to(hv.Curve, kdims=['session']).overlay('animal').layout('group').cols(1)

In [ ]:
hv.archive.export()